# Neo4j Lab 11: Python Driver & Service Architecture
## Part 3: Repository Pattern & Service Layer

**Duration:** 15 minutes  
**Objective:** Implement the repository pattern for data access and service layer for business logic

---

## Overview

This notebook covers:
- Abstract repository base class
- Customer repository implementation
- Repository pattern with CRUD operations
- Service layer with business logic
- Transaction management
- Complex business operations

## Cell 1: Import Dependencies

Import required modules and ensure previous notebooks are loaded.

In [ ]:
# Cell 1: Import dependencies
# Note: Make sure you've run notebooks 01 and 02 first

# If connection_manager and models are not available, uncomment and run:
# %run 01_python_driver_setup_and_basics.ipynb
# %run 02_pydantic_models_and_validation.ipynb

from abc import ABC, abstractmethod
from typing import Optional, List, Dict, Any, TypeVar, Generic, Callable
import json
import logging
from datetime import datetime, date, timedelta
import uuid

print("🏗️ IMPLEMENTING REPOSITORY PATTERN:")
print("=" * 50)

## Cell 2: Abstract Repository Base Class

Create an abstract base class that defines the interface for all repositories.

In [ ]:
# Cell 2: Abstract repository base class

T = TypeVar('T', bound=BaseModel)

class AbstractRepository(ABC, Generic[T]):
    """Abstract base repository for common database operations"""
    
    def __init__(self, connection_manager):
        self.connection_manager = connection_manager
        self.logger = logging.getLogger(self.__class__.__name__)
    
    @abstractmethod
    def create(self, entity: T) -> T:
        """Create a new entity"""
        pass
    
    @abstractmethod
    def get_by_id(self, entity_id: str) -> Optional[T]:
        """Get entity by ID"""
        pass
    
    @abstractmethod
    def update(self, entity: T) -> T:
        """Update existing entity"""
        pass
    
    @abstractmethod
    def delete(self, entity_id: str) -> bool:
        """Delete entity by ID"""
        pass
    
    @abstractmethod
    def list_all(self, limit: int = 100, offset: int = 0) -> List[T]:
        """List all entities with pagination"""
        pass
    
    def execute_query(self, query: str, parameters: Optional[Dict[str, Any]] = None) -> List[Dict[str, Any]]:
        """Execute raw query and return results"""
        try:
            records = self.connection_manager.execute_query(query, parameters)
            return [record.data() for record in records]
        except Exception as e:
            self.logger.error(f"Query execution failed: {e}")
            raise

print("✓ Abstract repository base class created")
print("  - Defines standard CRUD interface")
print("  - Generic type support")
print("  - Logging integration")
print("  - Query execution helper")

## Cell 3: Customer Repository Implementation

Implement a concrete repository for customer entities with full CRUD operations.

In [ ]:
# Cell 3: Customer repository implementation

class CustomerRepository(AbstractRepository[Customer]):
    """Repository for customer operations"""
    
    def create(self, customer: CustomerCreate) -> Customer:
        """Create a new customer in Neo4j"""
        query = """
        CREATE (c:Customer {
            customerId: $customer_id,
            firstName: $first_name,
            lastName: $last_name,
            email: $email,
            phone: $phone,
            dateOfBirth: date($date_of_birth),
            customerSince: date(),
            totalPolicies: 0,
            totalClaims: 0,
            customerValue: 0.0,
            riskScore: 50.0,
            initialContactMethod: $initial_contact_method,
            referralSource: $referral_source,
            createdAt: datetime(),
            updatedAt: datetime(),
            version: 1
        })
        RETURN c
        """
        
        parameters = {
            "customer_id": customer.customer_id,
            "first_name": customer.first_name,
            "last_name": customer.last_name,
            "email": customer.email,
            "phone": customer.phone,
            "date_of_birth": customer.date_of_birth.isoformat(),
            "initial_contact_method": customer.initial_contact_method,
            "referral_source": customer.referral_source
        }
        
        try:
            result = self.execute_query(query, parameters)
            if result:
                customer_data = result[0]['c']
                return Customer(**self._neo4j_to_dict(customer_data))
            else:
                raise Exception("Failed to create customer")
        except Exception as e:
            self.logger.error(f"Customer creation failed: {e}")
            raise
    
    def get_by_id(self, customer_id: str) -> Optional[Customer]:
        """Get customer by ID"""
        query = """
        MATCH (c:Customer {customerId: $customer_id})
        RETURN c
        """
        
        try:
            result = self.execute_query(query, {"customer_id": customer_id})
            if result:
                customer_data = result[0]['c']
                return Customer(**self._neo4j_to_dict(customer_data))
            return None
        except Exception as e:
            self.logger.error(f"Customer retrieval failed: {e}")
            raise
    
    def update(self, customer: Customer) -> Customer:
        """Update existing customer"""
        query = """
        MATCH (c:Customer {customerId: $customer_id})
        SET c.firstName = $first_name,
            c.lastName = $last_name,
            c.email = $email,
            c.phone = $phone,
            c.updatedAt = datetime(),
            c.version = c.version + 1
        RETURN c
        """
        
        parameters = {
            "customer_id": customer.customer_id,
            "first_name": customer.first_name,
            "last_name": customer.last_name,
            "email": customer.email,
            "phone": customer.phone
        }
        
        try:
            result = self.execute_query(query, parameters)
            if result:
                customer_data = result[0]['c']
                return Customer(**self._neo4j_to_dict(customer_data))
            else:
                raise Exception("Customer not found for update")
        except Exception as e:
            self.logger.error(f"Customer update failed: {e}")
            raise
    
    def delete(self, customer_id: str) -> bool:
        """Delete customer by ID"""
        query = """
        MATCH (c:Customer {customerId: $customer_id})
        DETACH DELETE c
        RETURN count(c) as deleted_count
        """
        
        try:
            result = self.execute_query(query, {"customer_id": customer_id})
            return result[0]['deleted_count'] > 0 if result else False
        except Exception as e:
            self.logger.error(f"Customer deletion failed: {e}")
            raise
    
    def list_all(self, limit: int = 100, offset: int = 0) -> List[Customer]:
        """List all customers with pagination"""
        query = """
        MATCH (c:Customer)
        RETURN c
        ORDER BY c.lastName, c.firstName
        SKIP $offset
        LIMIT $limit
        """
        
        try:
            result = self.execute_query(query, {"limit": limit, "offset": offset})
            return [Customer(**self._neo4j_to_dict(record['c'])) for record in result]
        except Exception as e:
            self.logger.error(f"Customer listing failed: {e}")
            raise
    
    def search_by_email(self, email: str) -> Optional[Customer]:
        """Search customer by email"""
        query = """
        MATCH (c:Customer {email: $email})
        RETURN c
        """
        
        try:
            result = self.execute_query(query, {"email": email})
            if result:
                customer_data = result[0]['c']
                return Customer(**self._neo4j_to_dict(customer_data))
            return None
        except Exception as e:
            self.logger.error(f"Customer email search failed: {e}")
            raise
    
    def get_customer_stats(self, customer_id: str) -> Dict[str, Any]:
        """Get comprehensive customer statistics"""
        query = """
        MATCH (c:Customer {customerId: $customer_id})
        OPTIONAL MATCH (c)-[:HOLDS]->(p:Policy)
        OPTIONAL MATCH (p)-[:COVERS]->(cl:Claim)
        RETURN c,
               count(DISTINCT p) as policy_count,
               count(DISTINCT cl) as claim_count,
               sum(p.premiumAmount) as total_premiums,
               sum(cl.claimAmount) as total_claims_amount
        """
        
        try:
            result = self.execute_query(query, {"customer_id": customer_id})
            if result:
                record = result[0]
                return {
                    "customer": Customer(**self._neo4j_to_dict(record['c'])),
                    "statistics": {
                        "policy_count": record['policy_count'] or 0,
                        "claim_count": record['claim_count'] or 0,
                        "total_premiums": float(record['total_premiums'] or 0),
                        "total_claims_amount": float(record['total_claims_amount'] or 0)
                    }
                }
            return None
        except Exception as e:
            self.logger.error(f"Customer stats retrieval failed: {e}")
            raise
    
    def _neo4j_to_dict(self, neo4j_node) -> Dict[str, Any]:
        """Convert Neo4j node to dictionary with proper type conversion"""
        data = dict(neo4j_node)
        
        # Convert Neo4j field names to Python model field names
        field_mapping = {
            'customerId': 'customer_id',
            'firstName': 'first_name',
            'lastName': 'last_name',
            'dateOfBirth': 'date_of_birth',
            'customerSince': 'customer_since',
            'totalPolicies': 'total_policies',
            'totalClaims': 'total_claims',
            'customerValue': 'customer_value',
            'riskScore': 'risk_score',
            'createdAt': 'created_at',
            'updatedAt': 'updated_at'
        }
        
        converted_data = {}
        for neo4j_key, value in data.items():
            python_key = field_mapping.get(neo4j_key, neo4j_key)
            converted_data[python_key] = value
        
        return converted_data

# Initialize repository
print("🏗️ INITIALIZING CUSTOMER REPOSITORY:")

try:
    customer_repo = CustomerRepository(connection_manager)
    print("✓ Customer repository initialized successfully")
    print("✓ Available operations:")
    print("  - create(): Create new customer")
    print("  - get_by_id(): Retrieve customer by ID")
    print("  - update(): Update existing customer")
    print("  - delete(): Delete customer")
    print("  - list_all(): List all customers with pagination")
    print("  - search_by_email(): Find customer by email")
    print("  - get_customer_stats(): Get comprehensive statistics")
    
except Exception as e:
    print(f"✗ Repository initialization failed: {e}")

print("=" * 50)

## Cell 4: Service Layer Implementation

Implement a service layer that orchestrates business logic using repositories.

In [ ]:
# Cell 4: Insurance service layer with business logic
print("🔧 IMPLEMENTING SERVICE LAYER:")
print("=" * 50)

class InsuranceService:
    """
    Service layer implementing insurance business logic
    Handles complex operations involving multiple entities
    """
    
    def __init__(self, connection_manager):
        self.connection_manager = connection_manager
        self.customer_repo = CustomerRepository(connection_manager)
        self.logger = logging.getLogger(self.__class__.__name__)
    
    def create_customer_with_policy(self, customer_data: CustomerCreate, policy_data: PolicyCreate) -> Dict[str, Any]:
        """Create customer and initial policy in a single transaction"""
        
        def create_transaction(tx):
            # Create customer
            customer_query = """
            CREATE (c:Customer {
                customerId: $customer_id,
                firstName: $first_name,
                lastName: $last_name,
                email: $email,
                phone: $phone,
                dateOfBirth: date($date_of_birth),
                customerSince: date(),
                totalPolicies: 1,
                totalClaims: 0,
                customerValue: $premium_amount,
                riskScore: 50.0,
                createdAt: datetime(),
                updatedAt: datetime(),
                version: 1
            })
            RETURN c
            """
            
            # Create policy
            policy_query = """
            MATCH (c:Customer {customerId: $customer_id})
            CREATE (p:Policy {
                policyNumber: $policy_number,
                policyType: $policy_type,
                customerId: $customer_id,
                effectiveDate: date($effective_date),
                expirationDate: date($expiration_date),
                premiumAmount: $premium_amount,
                coverageAmount: $coverage_amount,
                deductible: $deductible,
                policyStatus: 'Active',
                claimsCount: 0,
                totalClaimsAmount: 0.0,
                createdAt: datetime(),
                updatedAt: datetime(),
                version: 1
            })
            CREATE (c)-[:HOLDS]->(p)
            RETURN p
            """
            
            # Execute customer creation
            customer_result = tx.run(customer_query, {
                "customer_id": customer_data.customer_id,
                "first_name": customer_data.first_name,
                "last_name": customer_data.last_name,
                "email": customer_data.email,
                "phone": customer_data.phone,
                "date_of_birth": customer_data.date_of_birth.isoformat(),
                "premium_amount": policy_data.premium_amount
            })
            
            # Execute policy creation
            policy_result = tx.run(policy_query, {
                "customer_id": customer_data.customer_id,
                "policy_number": policy_data.policy_number,
                "policy_type": policy_data.policy_type.value,
                "effective_date": policy_data.effective_date.isoformat(),
                "expiration_date": policy_data.expiration_date.isoformat(),
                "premium_amount": policy_data.premium_amount,
                "coverage_amount": policy_data.coverage_amount,
                "deductible": policy_data.deductible or 0
            })
            
            customer_record = customer_result.single()
            policy_record = policy_result.single()
            
            return {
                "customer": dict(customer_record["c"]),
                "policy": dict(policy_record["p"])
            }
        
        try:
            result = self.connection_manager.execute_write_transaction(create_transaction)
            
            # Create audit record
            self._create_audit_record("customer_policy_creation", customer_data.customer_id)
            
            self.logger.info(f"Customer and policy created successfully: {customer_data.customer_id}")
            return result
            
        except Exception as e:
            self.logger.error(f"Customer/policy creation failed: {e}")
            raise Exception(f"Failed to create customer and policy: {e}")
    
    def process_claim(self, claim_data: ClaimCreate) -> Dict[str, Any]:
        """Process a new insurance claim with business logic"""
        
        # Validate policy exists and is active
        policy_check_query = """
        MATCH (p:Policy {policyNumber: $policy_number})
        WHERE p.policyStatus = 'Active'
        RETURN p.coverageAmount as coverage, p.deductible as deductible
        """
        
        try:
            policy_result = self.connection_manager.execute_query(
                policy_check_query, 
                {"policy_number": claim_data.policy_number}
            )
            
            if not policy_result:
                raise Exception(f"Policy {claim_data.policy_number} not found or inactive")
            
            policy_info = policy_result[0]
            coverage_amount = float(policy_info['coverage'])
            deductible = float(policy_info['deductible'])
            
            # Validate claim amount doesn't exceed coverage
            if claim_data.claim_amount > coverage_amount:
                raise Exception(f"Claim amount ${claim_data.claim_amount} exceeds coverage ${coverage_amount}")
            
            # Calculate potential payout (claim amount minus deductible)
            potential_payout = max(0, claim_data.claim_amount - deductible)
            
            # Create claim with business logic
            def create_claim_transaction(tx):
                create_claim_query = """
                MATCH (p:Policy {policyNumber: $policy_number})
                MATCH (c:Customer {customerId: p.customerId})
                CREATE (cl:Claim {
                    claimNumber: $claim_number,
                    policyNumber: $policy_number,
                    claimDate: date($claim_date),
                    incidentDate: date($incident_date),
                    claimAmount: $claim_amount,
                    description: $description,
                    claimStatus: 'Filed',
                    potentialPayout: $potential_payout,
                    priority: $priority,
                    createdAt: datetime(),
                    updatedAt: datetime(),
                    version: 1
                })
                CREATE (p)-[:COVERS]->(cl)
                
                // Update policy statistics
                SET p.claimsCount = p.claimsCount + 1,
                    p.totalClaimsAmount = p.totalClaimsAmount + $claim_amount,
                    p.updatedAt = datetime()
                
                // Update customer statistics  
                SET c.totalClaims = c.totalClaims + 1,
                    c.updatedAt = datetime()
                
                RETURN cl, p, c
                """
                
                result = tx.run(create_claim_query, {
                    "claim_number": claim_data.claim_number,
                    "policy_number": claim_data.policy_number,
                    "claim_date": claim_data.claim_date.isoformat(),
                    "incident_date": claim_data.incident_date.isoformat(),
                    "claim_amount": claim_data.claim_amount,
                    "description": claim_data.description,
                    "potential_payout": potential_payout,
                    "priority": getattr(claim_data, 'priority', 'Medium')
                })
                
                return result.single()
            
            result = self.connection_manager.execute_write_transaction(create_claim_transaction)
            
            # Create audit record
            self._create_audit_record("claim_creation", claim_data.claim_number)
            
            self.logger.info(f"Claim processed successfully: {claim_data.claim_number}")
            
            return {
                "claim": dict(result["cl"]),
                "policy": dict(result["p"]),
                "customer": dict(result["c"]),
                "business_analysis": {
                    "potential_payout": potential_payout,
                    "deductible_applied": deductible,
                    "coverage_utilization": (claim_data.claim_amount / coverage_amount) * 100
                }
            }
            
        except Exception as e:
            self.logger.error(f"Claim processing failed: {e}")
            raise
    
    def get_customer_360_view(self, customer_id: str) -> Dict[str, Any]:
        """Comprehensive customer view with all relationships"""
        query = """
        MATCH (c:Customer {customerId: $customer_id})
        OPTIONAL MATCH (c)-[:HOLDS]->(p:Policy)
        OPTIONAL MATCH (p)-[:COVERS]->(claim:Claim)
        OPTIONAL MATCH (c)-[:HAS_RISK_ASSESSMENT]->(ra:RiskAssessment)
        
        RETURN c,
               collect(DISTINCT p) as policies,
               collect(DISTINCT claim) as claims,
               collect(DISTINCT ra) as risk_assessments
        """
        
        try:
            result = self.connection_manager.execute_query(query, {"customer_id": customer_id})
            
            if not result:
                return {"error": "Customer not found"}
            
            data = result[0]
            customer_data = dict(data['c'])
            
            return {
                "customer": customer_data,
                "policies": [dict(p) for p in data['policies'] if p],
                "claims": [dict(c) for c in data['claims'] if c],
                "risk_assessments": [dict(ra) for ra in data['risk_assessments'] if ra],
                "summary": {
                    "total_policies": len([p for p in data['policies'] if p]),
                    "total_claims": len([c for c in data['claims'] if c]),
                    "latest_risk_score": customer_data.get('riskScore', 0)
                }
            }
            
        except Exception as e:
            self.logger.error(f"Customer 360 view failed: {e}")
            raise
    
    def _create_audit_record(self, action: str, entity_id: str):
        """Create audit trail record"""
        query = """
        CREATE (ar:AuditRecord {
            auditId: randomUUID(),
            action: $action,
            entityId: $entity_id,
            timestamp: datetime(),
            userId: 'system',
            details: 'Automated system action'
        })
        RETURN ar.auditId as audit_id
        """
        
        try:
            self.connection_manager.execute_query(query, {
                "action": action,
                "entity_id": entity_id
            })
        except Exception as e:
            self.logger.warning(f"Audit record creation failed: {e}")

# Initialize service
try:
    insurance_service = InsuranceService(connection_manager)
    print("✓ Insurance service initialized successfully")
    print("✓ Available service operations:")
    print("  - create_customer_with_policy(): Create customer and policy atomically")
    print("  - process_claim(): Process insurance claim with validation")
    print("  - get_customer_360_view(): Get comprehensive customer data")
    print("✓ Service layer implementation complete")
    
except Exception as e:
    print(f"✗ Service initialization failed: {e}")

print("=" * 50)

## Summary

In this notebook, you've:

1. ✅ Created an abstract repository base class with:
   - Generic type support
   - Standard CRUD interface
   - Logging integration
   - Query execution helpers

2. ✅ Implemented a concrete customer repository with:
   - Full CRUD operations (Create, Read, Update, Delete)
   - Advanced search capabilities
   - Statistics aggregation
   - Data type conversion between Neo4j and Python

3. ✅ Built a service layer that:
   - Orchestrates business logic
   - Manages transactions
   - Implements complex operations
   - Creates audit trails
   - Validates business rules

4. ✅ Demonstrated enterprise patterns:
   - Separation of concerns
   - Single Responsibility Principle
   - Dependency injection
   - Transaction management

**Next Steps:** Proceed to `04_testing_and_integration.ipynb` to implement comprehensive testing and integration tests.